This script is the main operator script for running functions through the az-tilt estimation pipeline. To test out a function, we run the following steps:

1) Collect a data set to test on.

2) Get metadata associated with the data set, as this metadata may be passed as parameters into the function (potentially not all metadata may be used, so this needs to be fleshed out as some metadata may be optional!)

3) Run function using the data set and the required metadata parameters and get outputs

In [17]:
# Import required packages

import pandas as pd
import glob 

In [18]:
# Read in all of the data sets and the associated metadata

data_files = glob.glob("./data/*.csv")
# Drop the metadata file
data_files = [x for x in data_files if 'metadata.csv' not in x]
print(data_files)

# Get the metadata file
metadata_df = pd.read_csv("./data/metadata.csv")

['./data\\1201_ac_power_hw__2801.csv', './data\\1332_inv1_ac_power__2642.csv', './data\\1332_inv2_ac_power__2646.csv', './data\\1332_inv3_ac_power__2650.csv', './data\\4_ac_power__315.csv']


In [ ]:
# Loop through each file, get its associated metadata, and run it through the function
for file in data_files:
    df = pd.read_csv(file, index_col=0, parse_dates=True)
    # Get the metadata associated with the specific file
    power_stream = file.split("\\")[-1].replace(".csv", '')
    print(power_stream)
    associated_metadata = dict(metadata_df[metadata_df['power_stream'] == power_stream].iloc[0])
    print(associated_metadata)
    # Run the data through the designated function
    
    

1201_ac_power_hw__2801
{'power_stream': '1201_ac_power_hw__2801', 'system_id': 1201, 'name': 'Distributed Sun - 5 Executive Campus', 'latitude': 39.9283, 'longitude': -75.0481, 'public': True, 'timezone': 'America/New_York'}
1332_inv1_ac_power__2642
{'power_stream': '1332_inv1_ac_power__2642', 'system_id': 1332, 'name': 'NREL Parking Garage', 'latitude': 39.7388, 'longitude': -105.1732, 'public': True, 'timezone': 'Etc/GMT+7'}
1332_inv2_ac_power__2646
{'power_stream': '1332_inv2_ac_power__2646', 'system_id': 1332, 'name': 'NREL Parking Garage', 'latitude': 39.7388, 'longitude': -105.1732, 'public': True, 'timezone': 'Etc/GMT+7'}
1332_inv3_ac_power__2650
{'power_stream': '1332_inv3_ac_power__2650', 'system_id': 1332, 'name': 'NREL Parking Garage', 'latitude': 39.7388, 'longitude': -105.1732, 'public': True, 'timezone': 'Etc/GMT+7'}


[]
